In [1]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/课程.txt', 'r') as file:
    content = file.read()

# Split the content by course entries
# Each course typically starts with "课程标题：" after a line number
course_blocks = re.split(r'\n\d+\|\n', content)
course_blocks = [block for block in course_blocks if "课程标题：" in block]

# Initialize lists to store courses
basic_courses = []
advanced_courses = []

# Track current category
current_category = None

# Process each line
lines = content.split('\n')
for line in lines:
    if "【基础课程】" in line:
        current_category = "basic"
    elif "【进阶课程】" in line:
        current_category = "advanced"

# Function to extract course information
def process_course_block(block):
    lines = block.split('\n')
    course_info = {}
    
    # Extract title
    title_match = re.search(r'课程标题：(.*)', block)
    if title_match:
        course_info['title'] = title_match.group(1).strip()
    
    # Extract description
    desc_match = re.search(r'课程简介：(.*)', block)
    if desc_match:
        course_info['description'] = desc_match.group(1).strip()
    
    # Extract chapters
    chapters = []
    for line in lines:
        chapter_match = re.search(r'- (第\d+章\s+.*)', line)
        if chapter_match:
            chapters.append(chapter_match.group(1).strip())
    
    course_info['chapters'] = chapters
    
    # Add default instructor info (since it's not provided in the data)
    course_info['instructor'] = {
        'name': '专业讲师',
        'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
        'avatar': f'/images/instructors/instructor_{len(processed_courses) + 1}.jpg'
    }
    
    # Add cover image (placeholder path)
    course_info['cover_image'] = f'/images/courses/{slugify(course_info["title"])}.jpg'
    
    # Add duration (placeholder)
    course_info['duration'] = f'{len(chapters) * 2}小时'
    
    # Add level
    if "基础课程" in current_category:
        course_info['level'] = '基础'
    else:
        course_info['level'] = '进阶'
        
    # Add category based on title keywords
    if any(keyword in course_info['title'].lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐']):
        course_info['category'] = '内容创作'
    elif any(keyword in course_info['title'].lower() for keyword in ['开发', 'saas', '应用']):
        course_info['category'] = '应用开发'
    elif any(keyword in course_info['title'].lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
        course_info['category'] = 'AI工具应用'
    elif any(keyword in course_info['title'].lower() for keyword in ['营销', '品牌', 'ip', '电商']):
        course_info['category'] = '数字营销'
    elif any(keyword in course_info['title'].lower() for keyword in ['办公', '效率']):
        course_info['category'] = '办公效率'
    else:
        course_info['category'] = '其他'
    
    return course_info

# Helper function to create URL-friendly slugs
def slugify(text):
    # Replace non-alphanumeric characters with hyphens
    slug = re.sub(r'[^\w\s]', '', text)
    slug = re.sub(r'\s+', '-', slug)
    return slug.lower()

# Process all courses
processed_courses = []
current_category = "基础课程"  # Default category

for block in content.split('\n\d+\|\n'):
    if "【基础课程】" in block:
        current_category = "基础课程"
        continue
    elif "【进阶课程】" in block:
        current_category = "进阶课程"
        continue
    
    if "课程标题：" in block:
        course = process_course_block(block)
        course['category_type'] = current_category
        processed_courses.append(course)

# Properly split the courses again using regex based on line numbers
courses = []
pattern = r'(\d+\|课程标题：.*?)(?=\d+\|课程标题：|\Z)'
course_blocks = re.findall(pattern, content, re.DOTALL)

for i, block in enumerate(course_blocks):
    # Determine if this is basic or advanced course
    is_advanced = False
    if "【进阶课程】" in content.split(block)[0].split("【基础课程】")[-1]:
        is_advanced = True
    
    # Extract title
    title_match = re.search(r'课程标题：(.*)', block)
    title = title_match.group(1).strip() if title_match else "未知课程"
    
    # Extract description
    desc_match = re.search(r'课程简介：(.*)', block)
    description = desc_match.group(1).strip() if desc_match else "无描述"
    
    # Extract chapters
    chapters = []
    chapter_matches = re.findall(r'- (第\d+章\s+.*)', block)
    for chapter in chapter_matches:
        chapters.append(chapter.strip())
    
    # Create course info dictionary
    course = {
        'id': i + 1,
        'title': title,
        'description': description,
        'chapters': chapters,
        'level': '进阶' if is_advanced else '基础',
        'instructor': {
            'name': '专业讲师',
            'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
            'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
        },
        'cover_image': f'/images/courses/course_{i+1}.jpg',
        'duration': f'{len(chapters) * 2}小时',
        'students_enrolled': 100 + (i * 50),
        'rating': round(4 + (i % 10) / 10, 1)
    }
    
    # Add category based on title keywords
    if any(keyword in title.lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
        course['category'] = '内容创作'
    elif any(keyword in title.lower() for keyword in ['开发', 'saas', '应用', '一人公司']):
        course['category'] = '应用开发'
    elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
        course['category'] = 'AI工具应用'
    elif any(keyword in title.lower() for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
        course['category'] = '数字营销'
    elif any(keyword in title.lower() for keyword in ['办公', '效率']):
        course['category'] = '办公效率'
    elif '法律' in title.lower():
        course['category'] = '法律与合规'
    else:
        course['category'] = '职业发展'
    
    courses.append(course)

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"Successfully processed {len(courses)} courses and saved to processed_courses.json")
print(f"Categories: {set(course['category'] for course in courses)}")
print(f"Levels: {set(course['level'] for course in courses)}")

Successfully processed 0 courses and saved to processed_courses.json
Categories: set()
Levels: set()


In [2]:
import json
import re

# Read the course data file again
with open('/data/chats/ejf70/workspace/uploads/课程.txt', 'r') as file:
    content = file.read()

# Better approach to extract courses
courses = []
current_category = "基础课程"  # Default starting category

# Split by line numbers
lines = content.split('\n')
course_data = None
course_title = None
course_desc = None
chapters = []

for line in lines:
    # Check for category markers
    if "【基础课程】" in line:
        current_category = "基础课程"
        continue
    elif "【进阶课程】" in line:
        current_category = "进阶课程"
        continue
    
    # Extract course title
    title_match = re.search(r'\d+\|课程标题：(.*)', line)
    if title_match:
        # If we already have a course in progress, save it before starting a new one
        if course_title:
            course = {
                'id': len(courses) + 1,
                'title': course_title,
                'description': course_desc,
                'chapters': chapters,
                'level': '进阶' if current_category == "进阶课程" else '基础',
                'instructor': {
                    'name': '专业讲师',
                    'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
                    'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
                },
                'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
                'duration': f'{len(chapters) * 2}小时',
                'students_enrolled': 100 + (len(courses) * 50),
                'rating': round(4 + (len(courses) % 10) / 10, 1)
            }
            
            # Add category based on title keywords
            if any(keyword in course_title.lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
                course['category'] = '内容创作'
            elif any(keyword in course_title.lower() for keyword in ['开发', 'saas', '应用', '一人公司']):
                course['category'] = '应用开发'
            elif any(keyword in course_title.lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
                course['category'] = 'AI工具应用'
            elif any(keyword in course_title.lower() for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
                course['category'] = '数字营销'
            elif any(keyword in course_title.lower() for keyword in ['办公', '效率']):
                course['category'] = '办公效率'
            elif '法律' in course_title.lower():
                course['category'] = '法律与合规'
            elif '教育' in course_title.lower():
                course['category'] = '教育应用'
            else:
                course['category'] = '职业发展'
                
            courses.append(course)
            
            # Reset for the next course
            chapters = []
        
        # Start collecting data for new course
        course_title = title_match.group(1).strip()
        
    # Extract course description
    desc_match = re.search(r'\d+\|课程简介：(.*)', line)
    if desc_match and course_title:  # Ensure we have a course title first
        course_desc = desc_match.group(1).strip()
        
    # Extract chapters
    chapter_match = re.search(r'\d+\|- (第\d+章\s+.*)', line)
    if chapter_match and course_title:  # Ensure we have a course title first
        chapters.append(chapter_match.group(1).strip())

# Add the last course if there is one in progress
if course_title:
    course = {
        'id': len(courses) + 1,
        'title': course_title,
        'description': course_desc,
        'chapters': chapters,
        'level': '进阶' if current_category == "进阶课程" else '基础',
        'instructor': {
            'name': '专业讲师',
            'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
            'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
        },
        'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
        'duration': f'{len(chapters) * 2}小时',
        'students_enrolled': 100 + (len(courses) * 50),
        'rating': round(4 + (len(courses) % 10) / 10, 1)
    }
    
    # Add category based on title keywords
    if any(keyword in course_title.lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
        course['category'] = '内容创作'
    elif any(keyword in course_title.lower() for keyword in ['开发', 'saas', '应用', '一人公司']):
        course['category'] = '应用开发'
    elif any(keyword in course_title.lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
        course['category'] = 'AI工具应用'
    elif any(keyword in course_title.lower() for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
        course['category'] = '数字营销'
    elif any(keyword in course_title.lower() for keyword in ['办公', '效率']):
        course['category'] = '办公效率'
    elif '法律' in course_title.lower():
        course['category'] = '法律与合规'
    elif '教育' in course_title.lower():
        course['category'] = '教育应用'
    else:
        course['category'] = '职业发展'
        
    courses.append(course)

# Add additional useful fields for the MasterClass style website
for course in courses:
    # Generate a URL-friendly slug
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Add a teaser text
    teaser_words = course['description'].split()[:15]
    course['teaser'] = ' '.join(teaser_words) + '...'
    
    # Add featured flag to some courses (for homepage highlights)
    course['featured'] = course['id'] % 3 == 0
    
    # Add difficulty level (for filtering)
    if len(course['chapters']) <= 5:
        course['difficulty'] = '初级'
    else:
        course['difficulty'] = '中级' if course['level'] == '基础' else '高级'
    
    # Add estimated completion time
    course['completion_time'] = f"{len(course['chapters']) * 3}天"
    
    # Add what you'll learn bullets
    course['what_youll_learn'] = [
        f"掌握{course['title'].split('用')[-1].split('与')[0] if '用' in course['title'] else course['title']}的核心技能",
        f"从零基础到能够独立完成{course['category']}相关项目",
        f"获得实战经验和行业洞察",
        f"学习最新的AI工具和技术应用方法"
    ]
    
    # Sample testimonials
    names = ["李明", "王华", "张伟", "赵敏", "陈静", "林小曼", "郭志强", "黄建国"]
    testimonials = [
        "这门课程让我找到了新的职业发展方向，非常实用！",
        "内容深入浅出，即使是零基础也能快速上手。",
        "讲师经验丰富，分享了很多实战技巧，值得推荐。",
        "课程设计很系统，学完后直接应用到了工作中。",
        "性价比超高的一门课，学到了很多实用技能。"
    ]
    
    course['testimonials'] = [
        {
            'name': names[i % len(names)],
            'comment': testimonials[i % len(testimonials)],
            'avatar': f'/images/testimonials/user_{i+1}.jpg',
            'rating': 5 if i % 3 == 0 else 4
        } for i in range(2)  # 2 testimonials per course
    ]

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"成功处理了 {len(courses)} 门课程并保存到 processed_courses.json")
print(f"课程类别: {set(course['category'] for course in courses)}")
print(f"课程级别: {set(course['level'] for course in courses)}")

成功处理了 0 门课程并保存到 processed_courses.json
课程类别: set()
课程级别: set()


In [3]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/课程.txt', 'r') as file:
    content = file.read()

# Extract basic and advanced course sections
basic_section_match = re.search(r'【基础课程】(.*?)【进阶课程】', content, re.DOTALL)
advanced_section_match = re.search(r'【进阶课程】(.*)', content, re.DOTALL)

basic_section = basic_section_match.group(1) if basic_section_match else ""
advanced_section = advanced_section_match.group(1) if advanced_section_match else ""

# Define a function to extract courses from a section
def extract_courses_from_section(section_text, level):
    courses = []
    # Match patterns for course blocks (start with course title and end before next course title or end of section)
    course_blocks = re.findall(r'\d+\|课程标题：(.*?)(?=\d+\|课程标题：|\Z)', section_text, re.DOTALL)
    
    # If no courses found with above pattern (which can happen), try another approach
    if not course_blocks:
        # Split by line numbers that contain course titles
        title_lines = re.findall(r'\d+\|课程标题：.*', section_text)
        # For each title line, find the corresponding block of text until the next title or end
        for i, title_line in enumerate(title_lines):
            if i < len(title_lines) - 1:
                next_title_idx = section_text.find(title_lines[i+1])
                course_block = section_text[section_text.find(title_line):next_title_idx]
            else:
                course_block = section_text[section_text.find(title_line):]
            course_blocks.append(course_block)
    
    for i, block in enumerate(course_blocks):
        # Extract title (from the first match of course title pattern)
        title_match = re.search(r'课程标题：(.*?)(?:\n|$)', block)
        title = title_match.group(1).strip() if title_match else f"课程 {i+1}"
        
        # Extract description
        desc_match = re.search(r'课程简介：(.*?)(?:\n|$)', block)
        description = desc_match.group(1).strip() if desc_match else "无课程简介"
        
        # Extract chapters
        chapters = []
        chapter_matches = re.findall(r'- (第\d+章.*?)(?:\n|$)', block)
        for chapter in chapter_matches:
            chapters.append(chapter.strip())
        
        # Create the course object with additional fields for MasterClass style
        course = {
            'id': len(courses) + 1,
            'title': title,
            'description': description,
            'chapters': chapters,
            'level': level,
            'instructor': {
                'name': '专业讲师',
                'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
                'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
            },
            'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
            'duration': f'{len(chapters) * 2}小时',
            'students_enrolled': 100 + (len(courses) * 50),
            'rating': round(4 + (len(courses) % 10) / 10, 1)
        }
        
        # Add category based on title keywords
        if any(keyword in title.lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
            course['category'] = '内容创作'
        elif any(keyword in title.lower() for keyword in ['开发', 'saas', '应用', '一人公司']):
            course['category'] = '应用开发'
        elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
            course['category'] = 'AI工具应用'
        elif any(keyword in title.lower() for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
            course['category'] = '数字营销'
        elif any(keyword in title.lower() for keyword in ['办公', '效率']):
            course['category'] = '办公效率'
        elif '法律' in title.lower():
            course['category'] = '法律与合规'
        elif '教育' in title.lower():
            course['category'] = '教育应用'
        else:
            course['category'] = '职业发展'
        
        # Generate a URL-friendly slug
        slug = re.sub(r'[^\w\s]', '', title)
        slug = re.sub(r'\s+', '-', slug).lower()
        course['slug'] = slug
        
        # Add a teaser text
        teaser_words = description.split()[:15] if description else []
        course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else "了解更多..."
        
        # Add featured flag to some courses (for homepage highlights)
        course['featured'] = (len(courses) % 3 == 0)
        
        # Add difficulty level (for filtering)
        if len(chapters) <= 5:
            course['difficulty'] = '初级'
        else:
            course['difficulty'] = '中级' if level == '基础' else '高级'
        
        # Add estimated completion time
        course['completion_time'] = f"{len(chapters) * 3}天"
        
        # Add what you'll learn bullets
        course['what_youll_learn'] = [
            f"掌握{title.split('用')[-1].split('与')[0] if '用' in title else title}的核心技能",
            f"从零基础到能够独立完成{course['category']}相关项目",
            f"获得实战经验和行业洞察",
            f"学习最新的AI工具和技术应用方法"
        ]
        
        # Sample testimonials
        names = ["李明", "王华", "张伟", "赵敏", "陈静", "林小曼", "郭志强", "黄建国"]
        testimonials = [
            "这门课程让我找到了新的职业发展方向，非常实用！",
            "内容深入浅出，即使是零基础也能快速上手。",
            "讲师经验丰富，分享了很多实战技巧，值得推荐。",
            "课程设计很系统，学完后直接应用到了工作中。",
            "性价比超高的一门课，学到了很多实用技能。"
        ]
        
        course['testimonials'] = [
            {
                'name': names[i % len(names)],
                'comment': testimonials[i % len(testimonials)],
                'avatar': f'/images/testimonials/user_{i+1}.jpg',
                'rating': 5 if i % 3 == 0 else 4
            } for i in range(2)  # 2 testimonials per course
        ]
        
        courses.append(course)
    
    return courses

# Extract courses from each section
basic_courses = extract_courses_from_section(basic_section, "基础")
advanced_courses = extract_courses_from_section(advanced_section, "进阶")

# Combine all courses
all_courses = []
for i, course in enumerate(basic_courses + advanced_courses):
    course['id'] = i + 1  # Reassign ids to ensure they're sequential
    all_courses.append(course)

# Try another approach if both methods failed to extract courses
if not all_courses:
    # Direct line-by-line parsing approach
    lines = content.split('\n')
    current_course = None
    current_level = "基础"  # Default
    courses = []
    
    for line in lines:
        if "【基础课程】" in line:
            current_level = "基础"
            continue
        elif "【进阶课程】" in line:
            current_level = "进阶"
            continue
        
        # Check for course title
        title_match = re.search(r'\d+\|课程标题：(.*)', line)
        if title_match:
            # Save previous course if exists
            if current_course:
                courses.append(current_course)
            
            # Start new course
            current_course = {
                'id': len(courses) + 1,
                'title': title_match.group(1).strip(),
                'description': '',
                'chapters': [],
                'level': current_level
            }
            continue
        
        # Add description
        desc_match = re.search(r'\d+\|课程简介：(.*)', line)
        if desc_match and current_course:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Add chapter
        chapter_match = re.search(r'\d+\|- (第\d+章\s+.*)', line)
        if chapter_match and current_course:
            current_course['chapters'].append(chapter_match.group(1).strip())
    
    # Add the last course
    if current_course:
        courses.append(current_course)
    
    # If we found courses this way, process them with additional fields
    if courses:
        all_courses = []
        for i, course in enumerate(courses):
            # Add instructor info
            course['instructor'] = {
                'name': '专业讲师',
                'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
                'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
            }
            
            # Add other fields
            course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
            course['duration'] = f'{len(course["chapters"]) * 2}小时'
            course['students_enrolled'] = 100 + (i * 50)
            course['rating'] = round(4 + (i % 10) / 10, 1)
            
            # Add category based on title
            title = course['title']
            if any(keyword in title.lower() for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
                course['category'] = '内容创作'
            elif any(keyword in title.lower() for keyword in ['开发', 'saas', '应用', '一人公司']):
                course['category'] = '应用开发'
            elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', '智能助手']):
                course['category'] = 'AI工具应用'
            elif any(keyword in title.lower() for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
                course['category'] = '数字营销'
            elif any(keyword in title.lower() for keyword in ['办公', '效率']):
                course['category'] = '办公效率'
            elif '法律' in title.lower():
                course['category'] = '法律与合规'
            elif '教育' in title.lower():
                course['category'] = '教育应用'
            else:
                course['category'] = '职业发展'
                
            # Generate slug
            slug = re.sub(r'[^\w\s]', '', title)
            slug = re.sub(r'\s+', '-', slug).lower()
            course['slug'] = slug
            
            # Add other MasterClass style fields
            course['teaser'] = course['description'][:50] + '...' if course['description'] else "了解更多..."
            course['featured'] = (i % 3 == 0)
            course['difficulty'] = '初级' if len(course['chapters']) <= 5 else ('中级' if course['level'] == '基础' else '高级')
            course['completion_time'] = f"{len(course['chapters']) * 3}天"
            course['what_youll_learn'] = [
                f"掌握{title.split('用')[-1].split('与')[0] if '用' in title else title}的核心技能",
                f"从零基础到能够独立完成{course['category']}相关项目",
                f"获得实战经验和行业洞察",
                f"学习最新的AI工具和技术应用方法"
            ]
            
            # Add testimonials
            names = ["李明", "王华", "张伟", "赵敏", "陈静", "林小曼", "郭志强", "黄建国"]
            testimonials = [
                "这门课程让我找到了新的职业发展方向，非常实用！",
                "内容深入浅出，即使是零基础也能快速上手。",
                "讲师经验丰富，分享了很多实战技巧，值得推荐。",
                "课程设计很系统，学完后直接应用到了工作中。",
                "性价比超高的一门课，学到了很多实用技能。"
            ]
            
            course['testimonials'] = [
                {
                    'name': names[j % len(names)],
                    'comment': testimonials[j % len(testimonials)],
                    'avatar': f'/images/testimonials/user_{j+1}.jpg',
                    'rating': 5 if j % 3 == 0 else 4
                } for j in range(2)  # 2 testimonials per course
            ]
            
            all_courses.append(course)

# Save the processed courses to JSON
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(all_courses, out_file, ensure_ascii=False, indent=2)

print(f"成功处理了 {len(all_courses)} 门课程并保存到 processed_courses.json")
if all_courses:
    print(f"课程类别: {set(course['category'] for course in all_courses)}")
    print(f"课程级别: {set(course['level'] for course in all_courses)}")

成功处理了 0 门课程并保存到 processed_courses.json


In [4]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/课程.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# Initialize lists to store courses
courses = []
current_level = "基础"  # Default level
current_course = None

# Process line by line
lines = content.split('\n')
for line in lines:
    # Check for level markers
    if "【基础课程】" in line:
        current_level = "基础"
        continue
    elif "【进阶课程】" in line:
        current_level = "进阶"
        continue
    
    # Check for course title
    title_match = re.search(r'\d+\|课程标题：(.*)', line)
    if title_match:
        # Save previous course if exists
        if current_course:
            courses.append(current_course)
        
        # Start new course
        current_course = {
            'id': len(courses) + 1,
            'title': title_match.group(1).strip(),
            'description': '',
            'chapters': [],
            'level': current_level
        }
        continue
    
    # Check for course description
    if current_course:  # Only proceed if we have a current course
        desc_match = re.search(r'\d+\|课程简介：(.*)', line)
        if desc_match:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Check for chapters
        chapter_match = re.search(r'\d+\|- (第\d+章.*)', line)
        if chapter_match:
            current_course['chapters'].append(chapter_match.group(1).strip())

# Add the last course if there is one
if current_course:
    courses.append(current_course)

# Enhance course data with additional fields for MasterClass style
for i, course in enumerate(courses):
    # Add instructor information
    course['instructor'] = {
        'name': f'专业讲师{i+1}',
        'bio': '拥有丰富的AI应用实战经验，专注于帮助学员掌握AI工具的实际应用',
        'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
    }
    
    # Add course metadata
    course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
    course['duration'] = f'{len(course["chapters"]) * 2}小时'
    course['students_enrolled'] = 100 + (i * 50)
    course['rating'] = round(4 + (i % 10) / 10, 1)
    
    # Add category based on title keywords
    title = course['title'].lower()
    if any(keyword in title for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
        course['category'] = '内容创作'
    elif any(keyword in title for keyword in ['开发', 'saas', '应用', '一人公司']):
        course['category'] = '应用开发'
    elif any(keyword in title for keyword in ['chatgpt', 'prompt', '智能助手']):
        course['category'] = 'AI工具应用'
    elif any(keyword in title for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
        course['category'] = '数字营销'
    elif any(keyword in title for keyword in ['办公', '效率']):
        course['category'] = '办公效率'
    elif '法律' in title:
        course['category'] = '法律与合规'
    elif '教育' in title:
        course['category'] = '教育应用'
    else:
        course['category'] = '职业发展'
    
    # Create URL-friendly slug
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Add teaser
    teaser_words = course['description'].split()[:15]
    course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else '了解更多...'
    
    # Add featured flag
    course['featured'] = (i % 3 == 0)
    
    # Add difficulty level
    if len(course['chapters']) <= 5:
        course['difficulty'] = '初级'
    else:
        course['difficulty'] = '中级' if course['level'] == '基础' else '高级'
    
    # Add completion time estimate
    course['completion_time'] = f"{len(course['chapters']) * 3}天"
    
    # Add learning objectives
    course['what_youll_learn'] = [
        f"掌握{course['title'].split('用')[-1].split('与')[0] if '用' in course['title'] else course['title']}的核心技能",
        f"从零基础到能够独立完成{course['category']}相关项目",
        f"获得实战经验和行业洞察",
        f"学习最新的AI工具和技术应用方法"
    ]
    
    # Add testimonials
    names = ["李明", "王华", "张伟", "赵敏", "陈静", "林小曼", "郭志强", "黄建国"]
    testimonials = [
        "这门课程让我找到了新的职业发展方向，非常实用！",
        "内容深入浅出，即使是零基础也能快速上手。",
        "讲师经验丰富，分享了很多实战技巧，值得推荐。",
        "课程设计很系统，学完后直接应用到了工作中。",
        "性价比超高的一门课，学到了很多实用技能。"
    ]
    
    course['testimonials'] = [
        {
            'name': names[j % len(names)],
            'comment': testimonials[j % len(testimonials)],
            'avatar': f'/images/testimonials/user_{j+1}.jpg',
            'rating': 5 if j % 3 == 0 else 4
        } for j in range(2)  # 2 testimonials per course
    ]

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"成功处理了 {len(courses)} 门课程并保存到 processed_courses.json")
if courses:
    print(f"课程类别: {set(course['category'] for course in courses)}")
    print(f"课程级别: {set(course['level'] for course in courses)}")

成功处理了 0 门课程并保存到 processed_courses.json


In [5]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/课程.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# Initialize variables for parsing
courses = []
current_level = "基础"  # Default starting level
current_course = None

# Parse the file content
lines = content.split('\n')
for line in lines:
    # Check for category headers
    if "【基础课程】" in line:
        current_level = "基础"
        continue
    elif "【进阶课程】" in line:
        current_level = "进阶"
        continue
    
    # Extract course title - remove line numbers if present
    title_match = re.search(r'(?:\d+\|)?课程标题：(.*)', line)
    if title_match:
        # Save previous course if exists
        if current_course:
            courses.append(current_course)
        
        # Start new course
        current_course = {
            'id': len(courses) + 1,
            'title': title_match.group(1).strip(),
            'description': '',
            'chapters': [],
            'level': current_level
        }
        continue
    
    # Process only if we have a current course
    if current_course:
        # Extract course description - remove line numbers if present
        desc_match = re.search(r'(?:\d+\|)?课程简介：(.*)', line)
        if desc_match:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Extract chapter information - remove line numbers if present
        chapter_match = re.search(r'(?:\d+\|)?- (第\d+章.*)', line)
        if chapter_match:
            current_course['chapters'].append(chapter_match.group(1).strip())

# Add the last course if there is one
if current_course:
    courses.append(current_course)

# Enhance course data with additional fields for MasterClass style
for i, course in enumerate(courses):
    # Add instructor information - using realistic Chinese names for instructors
    instructor_names = ["李明远", "王志强", "张天琪", "陈思远", "林雨晴"]
    instructor_bios = [
        "AI应用专家，10年互联网产品经验，专注AI+个人品牌领域研究",
        "前大厂技术总监，AI开发实战派，擅长将复杂技术简化落地",
        "资深内容创作者，AI创意应用专家，曾帮助上百企业完成AI转型",
        "电商运营专家，精通AI营销策略，《AI商业应用》作者",
        "办公效率专家，AI工具集成应用权威，企业AI培训讲师"
    ]
    
    course['instructor'] = {
        'name': instructor_names[i % len(instructor_names)],
        'bio': instructor_bios[i % len(instructor_bios)],
        'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
    }
    
    # Add course metadata
    course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
    course['duration'] = f'{len(course["chapters"]) * 2}小时'
    course['students_enrolled'] = 100 + (i * 50)
    course['rating'] = round(4.5 + (i % 10) / 20, 1)  # Ratings between 4.5-5.0
    
    # Add category based on title keywords - more precise categorization
    title = course['title'].lower()
    if any(keyword in title for keyword in ['短视频', '图文', '漫画', '创作', '音乐', '剧本']):
        course['category'] = '内容创作'
    elif any(keyword in title for keyword in ['开发', 'saas', '应用', '一人公司']):
        course['category'] = '应用开发'
    elif any(keyword in title for keyword in ['chatgpt', 'prompt', '智能助手']):
        course['category'] = 'AI工具应用'
    elif any(keyword in title for keyword in ['营销', '品牌', 'ip', '电商', '文案']):
        course['category'] = '数字营销'
    elif any(keyword in title for keyword in ['办公', '效率']):
        course['category'] = '办公效率'
    elif '法律' in title:
        course['category'] = '法律与合规'
    elif '教育' in title:
        course['category'] = '教育应用'
    else:
        course['category'] = '职业发展'
    
    # Create URL-friendly slug
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Add teaser - more engaging preview
    teaser_words = course['description'].split()[:12]
    course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else '探索AI如何改变你的职业生涯...'
    
    # Add featured flag - distribute featured courses more evenly
    course['featured'] = (i % 4 == 0 or i % 7 == 0)  # More interesting distribution
    
    # Add difficulty level - based on course content and level
    chapter_count = len(course['chapters'])
    if chapter_count <= 5:
        course['difficulty'] = '入门级'
    else:
        course['difficulty'] = '进阶级' if course['level'] == '进阶' else '基础级'
    
    # Add completion time estimate - more realistic estimates
    course['completion_time'] = f"{chapter_count * 2 + 5}天"
    
    # Add learning objectives - more specific to each course
    title_focus = course['title'].split('用')[-1].split('与')[0] if '用' in course['title'] else course['title']
    course['what_youll_learn'] = [
        f"掌握{title_focus}的核心技能与工作流程",
        f"从零基础到能够独立完成{course['category']}实际项目",
        f"获得行业前沿洞察和实战经验",
        f"构建属于自己的AI工作方法论"
    ]
    
    # Add testimonials - more personalized
    names = ["李晓华", "王建明", "张思琪", "赵云峰", "陈明亮", "林小曼", "郭志强", "黄建国"]
    jobs = ["产品经理", "自由职业者", "内容创作者", "市场营销", "开发工程师", "企业老板", "大学教师", "设计师"]
    
    testimonial_templates = [
        "这门课程让我{outcome}，{benefit}！",
        "{timeframe}就掌握了{skill}，{result}。",
        "讲师{quality}，{learning}，{recommendation}。",
        "课程{feature}，{application}。",
        "性价比超高！{before}，现在{after}。"
    ]
    
    outcomes = [
        "找到了新的职业方向",
        "效率提升了3倍",
        "解决了长期困扰我的问题",
        "获得了实际可用的技能",
        "打开了AI应用的新视野"
    ]
    
    benefits = [
        "非常实用",
        "值得每一分钱",
        "强烈推荐给同行",
        "学以致用",
        "投资回报率极高"
    ]
    
    course['testimonials'] = []
    for j in range(2):  # 2 testimonials per course
        idx = (i + j) % len(names)
        testimonial = {
            'name': names[idx],
            'job': jobs[idx],
            'comment': f"这门{course['title']}课程非常出色！{outcomes[idx % len(outcomes)]}，{benefits[j % len(benefits)]}。",
            'avatar': f'/images/testimonials/user_{idx+1}.jpg',
            'rating': 5 if idx % 3 == 0 else 4
        }
        course['testimonials'].append(testimonial)
        
    # Add pricing information for MasterClass style
    course['pricing'] = {
        'original': 399 + (i % 5) * 100,
        'current': 299 + (i % 5) * 100,
        'discount_percent': 25 if i % 3 == 0 else 0
    }
    
    # Add trailer video (placeholder)
    course['trailer_video'] = f'/videos/courses/course_{i+1}_trailer.mp4'
    
    # Add related skills tags
    skill_tags = {
        '内容创作': ['创意写作', 'AI生成内容', '视觉设计', '短视频制作', '内容策划'],
        '应用开发': ['无代码开发', 'API集成', '用户体验', '产品设计', '自动化'],
        'AI工具应用': ['提示工程', '大语言模型', '知识库构建', 'AI工作流', '自动化脚本'],
        '数字营销': ['内容营销', '社交媒体', '广告创意', '用户增长', '数据分析'],
        '办公效率': ['工作流优化', '自动化报表', '团队协作', '项目管理', 'AI助手'],
        '法律与合规': ['数据合规', '知识产权', 'AI伦理', '法律风险', '隐私保护'],
        '教育应用': ['个性化学习', '教学设计', '学习评估', '内容开发', '互动体验'],
        '职业发展': ['技能提升', '个人品牌', '自由职业', '副业发展', '职业规划']
    }
    
    course['skill_tags'] = skill_tags.get(course['category'], [])[:3]

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"成功处理了 {len(courses)} 门课程并保存到 processed_courses.json")
if courses:
    print(f"课程类别: {set(course['category'] for course in courses)}")
    print(f"课程级别: {set(course['level'] for course in courses)}")
    print("示例课程：")
    print(json.dumps(courses[0], ensure_ascii=False, indent=2))

成功处理了 17 门课程并保存到 processed_courses.json
课程类别: {'AI工具应用', '应用开发', '内容创作', '职业发展', '法律与合规', '办公效率', '数字营销'}
课程级别: {'进阶', '基础'}
示例课程：
{
  "id": 1,
  "title": "用AI快速打造个人品牌",
  "description": "学习如何使用AI工具高效建立个人品牌，从头像制作到内容输出，轻松塑造专业形象。",
  "chapters": [
    "第1章 什么是个人品牌与AI结合？",
    "第2章 用Midjourney制作高质量头像",
    "第3章 ChatGPT帮你写自我介绍与个人故事",
    "第4章 Canva+AI生成社交媒体内容",
    "第5章 快速搭建个人网站（零代码）",
    "第6章 项目实操：打造你的第一个AI个人品牌案例"
  ],
  "level": "基础",
  "instructor": {
    "name": "李明远",
    "bio": "AI应用专家，10年互联网产品经验，专注AI+个人品牌领域研究",
    "avatar": "/images/instructors/instructor_1.jpg"
  },
  "cover_image": "/images/courses/course_1.jpg",
  "duration": "12小时",
  "students_enrolled": 100,
  "rating": 4.5,
  "category": "数字营销",
  "slug": "用ai快速打造个人品牌",
  "teaser": "学习如何使用AI工具高效建立个人品牌，从头像制作到内容输出，轻松塑造专业形象。...",
  "featured": true,
  "difficulty": "基础级",
  "completion_time": "17天",
  "what_youll_learn": [
    "掌握AI快速打造个人品牌的核心技能与工作流程",
    "从零基础到能够独立完成数字营销实际项目",
    "获得行业前沿洞察和实战经验",
    "构建属于自己的AI工作方法论"
  ],
 

In [6]:
I'll create a MasterClass-style website homepage for the online course subscription platform. The design will feature a dark elegant theme with high-quality visuals and clean typography.

```python
import json
from pathlib import Path

# Read the processed course data
with open('/data/chats/ejf70/workspace/processed_courses.json', 'r', encoding='utf-8') as file:
    courses = json.load(file)

# Create directories for the website if they don't exist
Path('/data/chats/ejf70/workspace/website').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/css').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/js').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/images').mkdir(exist_ok=True)

# Sort and organize courses
featured_courses = [course for course in courses if course.get('featured', False)]
basic_courses = [course for course in courses if course['level'] == '基础']
advanced_courses = [course for course in courses if course['level'] == '进阶']

# Helper function to get courses by category
def get_courses_by_category(category, limit=4):
    return [c for c in courses if c.get('category') == category][:limit]

# Create HTML file
html_content = """
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI学堂 - 在线AI课程订阅平台</title>
    <link rel="stylesheet" href="css/styles.css">
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans+SC:wght@300;400;500;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
</head>
<body>
    <div class="preloader">
        <div class="preloader-content">
            <div class="logo">AI<span>学堂</span></div>
            <div class="spinner"></div>
        </div>
    </div>

    <header class="main-header">
        <div class="header-container">
            <div class="logo">AI<span>学堂</span></div>
            <nav class="main-nav">
                <ul class="nav-links">
                    <li><a href="#featured" class="nav-link">精选课程</a></li>
                    <li><a href="#categories" class="nav-link">课程分类</a></li>
                    <li><a href="#instructors" class="nav-link">讲师团队</a></li>
                    <li><a href="#testimonials" class="nav-link">学员评价</a></li>
                </ul>
            </nav>
            <div class="header-actions">
                <div class="search-container">
                    <input type="text" placeholder="搜索课程..." class="search-input">
                    <button class="search-btn"><i class="fas fa-search"></i></button>
                </div>
                <a href="#" class="btn btn-outline">登录</a>
                <a href="#" class="btn btn-primary">开始学习</a>
            </div>
            <button class="mobile-menu-btn">
                <span></span><span></span><span></span>
            </button>
        </div>
    </header>

    <main>
        <!-- Hero Section -->
        <section class="hero-section">
            <div class="hero-content">
                <h1 class="hero-title">用AI提升<br>你的职业竞争力</h1>
                <p class="hero-subtitle">从基础应用到进阶技能，探索AI如何改变你的工作与生活</p>
                <div class="hero-cta">
                    <a href="#featured" class="btn btn-primary btn-large">浏览课程</a>
                    <a href="#" class="btn btn-outline btn-large">了解会员</a>
                </div>
                <div class="hero-stats">
                    <div class="stat-item">
                        <span class="stat-number">17+</span>
                        <span class="stat-label">精品课程</span>
                    </div>
                    <div class="stat-item">
                        <span class="stat-number">7</span>
                        <span class="stat-label">专业领域</span>
                    </div>
                    <div class="stat-item">
                        <span class="stat-number">5000+</span>
                        <span class="stat-label">学员</span>
                    </div>
                </div>
            </div>
            <div class="hero-image-container">
                <div class="hero-image"></div>
                <div class="floating-card card-1">
                    <div class="card-icon"><i class="fas fa-brain"></i></div>
                    <div class="card-text">AI思维训练</div>
                </div>
                <div class="floating-card card-2">
                    <div class="card-icon"><i class="fas fa-rocket"></i></div>
                    <div class="card-text">效率提升10倍</div>
                </div>
            </div>
        </section>

        <!-- Featured Courses Section -->
        <section id="featured" class="featured-section section">
            <div class="section-header">
                <h2 class="section-title">精选课程</h2>
                <p class="section-subtitle">由专业讲师精心设计的热门AI应用课程</p>
            </div>
            <div class="courses-grid">
"""

# Add featured courses
for course in featured_courses[:4]:  # Limit to 4 featured courses
    html_content += f"""
                <div class="course-card featured">
                    <div class="course-image-container">
                        <div class="course-image" style="background-image: url('{course['cover_image']}');"></div>
                        <div class="course-level">{course['level']}</div>
                        <div class="course-duration"><i class="far fa-clock"></i> {course['duration']}</div>
                    </div>
                    <div class="course-content">
                        <div class="course-category">{course['category']}</div>
                        <h3 class="course-title">{course['title']}</h3>
                        <p class="course-description">{course['teaser']}</p>
                        <div class="course-meta">
                            <div class="course-instructor">
                                <img src="{course['instructor']['avatar']}" alt="{course['instructor']['name']}" class="instructor-avatar">
                                <span>{course['instructor']['name']}</span>
                            </div>
                            <div class="course-rating">
                                <span class="rating-score">{course['rating']}</span>
                                <div class="rating-stars">
                                    {"".join(['<i class="fas fa-star"></i>' for _ in range(int(course['rating']))])}
                                    {"<i class='fas fa-star-half-alt'></i>" if course['rating'] % 1 >= 0.5 else ""}
                                </div>
                            </div>
                        </div>
                    </div>
                    <a href="#" class="course-link" data-id="{course['id']}">查看详情</a>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Categories Section -->
        <section id="categories" class="categories-section section">
            <div class="section-header">
                <h2 class="section-title">课程分类</h2>
                <p class="section-subtitle">按照你的学习目标和兴趣探索课程</p>
            </div>
            <div class="categories-tabs">
                <div class="tabs-header">
                    <button class="tab-btn active" data-tab="ai-tools">AI工具应用</button>
                    <button class="tab-btn" data-tab="content">内容创作</button>
                    <button class="tab-btn" data-tab="development">应用开发</button>
                    <button class="tab-btn" data-tab="marketing">数字营销</button>
                    <button class="tab-btn" data-tab="efficiency">办公效率</button>
                </div>
                
                <div class="tabs-content">
"""

# AI Tools Category Tab
ai_tools_courses = get_courses_by_category("AI工具应用")
html_content += """
                    <div class="tab-pane active" id="ai-tools">
                        <div class="category-info">
                            <div class="category-description">
                                <h3>AI工具应用</h3>
                                <p>掌握ChatGPT、Midjourney等前沿AI工具的实用技巧，无需编程基础，让AI成为你的得力助手。</p>
                                <ul class="category-features">
                                    <li><i class="fas fa-check"></i> 提示词工程与高级应用</li>
                                    <li><i class="fas fa-check"></i> 工作流程自动化</li>
                                    <li><i class="fas fa-check"></i> 个性化AI助手构建</li>
                                </ul>
                                <a href="#" class="btn btn-outline">浏览全部</a>
                            </div>
                            <div class="category-courses">
"""

for course in ai_tools_courses:
    html_content += f"""
                                <div class="category-course-card">
                                    <div class="category-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                    <div class="category-course-content">
                                        <h4>{course['title']}</h4>
                                        <div class="category-course-meta">
                                            <span>{course['duration']}</span>
                                            <span>{course['level']}</span>
                                        </div>
                                    </div>
                                </div>
    """

html_content += """
                            </div>
                        </div>
                    </div>
"""

# Content Creation Category Tab
content_courses = get_courses_by_category("内容创作")
html_content += """
                    <div class="tab-pane" id="content">
                        <div class="category-info">
                            <div class="category-description">
                                <h3>内容创作</h3>
                                <p>掌握AI内容生成工具，轻松创建短视频、图文、音频等多种形式的创意内容，提升创作效率。</p>
                                <ul class="category-features">
                                    <li><i class="fas fa-check"></i> AI图像与视频创作</li>
                                    <li><i class="fas fa-check"></i> 自动化内容分发</li>
                                    <li><i class="fas fa-check"></i> 创意IP打造</li>
                                </ul>
                                <a href="#" class="btn btn-outline">浏览全部</a>
                            </div>
                            <div class="category-courses">
"""

for course in content_courses:
    html_content += f"""
                                <div class="category-course-card">
                                    <div class="category-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                    <div class="category-course-content">
                                        <h4>{course['title']}</h4>
                                        <div class="category-course-meta">
                                            <span>{course['duration']}</span>
                                            <span>{course['level']}</span>
                                        </div>
                                    </div>
                                </div>
    """

html_content += """
                            </div>
                        </div>
                    </div>
"""

# Add other category tabs placeholders
html_content += """
                    <div class="tab-pane" id="development">
                        <!-- Development courses tab content will be populated by JavaScript -->
                    </div>
                    <div class="tab-pane" id="marketing">
                        <!-- Marketing courses tab content will be populated by JavaScript -->
                    </div>
                    <div class="tab-pane" id="efficiency">
                        <!-- Efficiency courses tab content will be populated by JavaScript -->
                    </div>
                </div>
            </div>
        </section>

        <!-- Learning Path Section -->
        <section class="learning-path-section section">
            <div class="section-header">
                <h2 class="section-title">学习路径</h2>
                <p class="section-subtitle">从零基础到专业应用，我们设计了清晰的学习路径</p>
            </div>
            <div class="learning-path-container">
                <div class="path-navigator">
                    <div class="path-node active">
                        <div class="node-number">01</div>
                        <div class="node-content">
                            <h3>基础入门</h3>
                            <p>了解AI基础概念与工具</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">02</div>
                        <div class="node-content">
                            <h3>实用应用</h3>
                            <p>掌握AI在特定场景中的应用</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">03</div>
                        <div class="node-content">
                            <h3>进阶技能</h3>
                            <p>深入学习高级技术与方法</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">04</div>
                        <div class="node-content">
                            <h3>专业实战</h3>
                            <p>真实项目实操与成果产出</p>
                        </div>
                    </div>
                </div>
                <div class="path-content">
                    <div class="path-detail active">
                        <h3>AI基础入门</h3>
                        <p>适合零基础学员，了解AI工具的基本概念、工作原理与常见应用场景，建立AI思维基础。</p>
                        <div class="path-courses">
"""

# Add basic level courses to the first path node
for course in basic_courses[:3]:
    html_content += f"""
                            <div class="path-course">
                                <div class="path-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                <div class="path-course-info">
                                    <h4>{course['title']}</h4>
                                    <div class="path-course-meta">
                                        <span>{course['duration']}</span>
                                        <span class="difficulty">{course['difficulty']}</span>
                                    </div>
                                </div>
                            </div>
    """

html_content += """
                        </div>
                        <a href="#" class="btn btn-primary">开始学习</a>
                    </div>
                    <!-- Other path details will be shown/hidden via JavaScript -->
                </div>
            </div>
        </section>

        <!-- Instructors Section -->
        <section id="instructors" class="instructors-section section">
            <div class="section-header">
                <h2 class="section-title">顶尖讲师团队</h2>
                <p class="section-subtitle">来自行业一线的专业讲师，带你掌握实战技能</p>
            </div>
            <div class="instructors-container">
"""

# Get unique instructors
unique_instructors = {}
for course in courses:
    instructor = course['instructor']
    if instructor['name'] not in unique_instructors:
        unique_instructors[instructor['name']] = instructor

# Add instructors
for name, instructor in list(unique_instructors.items())[:5]:  # Limit to 5 instructors
    html_content += f"""
                <div class="instructor-card">
                    <div class="instructor-image" style="background-image: url('{instructor['avatar']}');"></div>
                    <h3 class="instructor-name">{instructor['name']}</h3>
                    <p class="instructor-title">{instructor['bio'].split('，')[0]}</p>
                    <p class="instructor-bio">{instructor['bio']}</p>
                    <div class="instructor-stats">
                        <div class="stat">
                            <span class="stat-number">{courses.count(instructor['name'])}</span>
                            <span class="stat-label">课程</span>
                        </div>
                        <div class="stat">
                            <span class="stat-number">4.8</span>
                            <span class="stat-label">评分</span>
                        </div>
                        <div class="stat">
                            <span class="stat-number">1000+</span>
                            <span class="stat-label">学员</span>
                        </div>
                    </div>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Testimonials Section -->
        <section id="testimonials" class="testimonials-section section">
            <div class="section-header">
                <h2 class="section-title">学员反馈</h2>
                <p class="section-subtitle">看看其他学员如何通过我们的课程提升技能</p>
            </div>
            <div class="testimonials-slider">
"""

# Collect testimonials from different courses
all_testimonials = []
for course in courses:
    if 'testimonials' in course:
        for testimonial in course['testimonials']:
            testimonial['course_title'] = course['title']
            all_testimonials.append(testimonial)

# Add testimonials
for testimonial in all_testimonials[:6]:  # Limit to 6 testimonials
    html_content += f"""
                <div class="testimonial-card">
                    <div class="testimonial-header">
                        <div class="testimonial-avatar" style="background-image: url('{testimonial['avatar']}');"></div>
                        <div class="testimonial-author">
                            <h4 class="author-name">{testimonial['name']}</h4>
                            <p class="author-title">{testimonial.get('job', '学员')}</p>
                        </div>
                        <div class="testimonial-rating">
                            {"".join(['<i class="fas fa-star"></i>' for _ in range(testimonial['rating'])])}
                        </div>
                    </div>
                    <div class="testimonial-content">
                        <p>"{testimonial['comment']}"</p>
                        <div class="testimonial-course">— 学习了 {testimonial['course_title']}</div>
                    </div>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Subscription Section -->
        <section class="subscription-section">
            <div class="subscription-content">
                <h2>成为会员，解锁全部课程</h2>
                <p>一次订阅，获取所有课程和未来更新，随时学习，无限次观看</p>
                <div class="subscription-plans">
                    <div class="plan-card">
                        <div class="plan-header">
                            <h3>月度会员</h3>
                            <div class="plan-price">
                                <span class="currency">¥</span>
                                <span class="amount">99</span>
                                <span class="period">/月</span>
                            </div>
                        </div>
                        <ul class="plan-features">
                            <li><i class="fas fa-check"></i> 无限访问所有课程</li>
                            <li><i class="fas fa-check"></i> 随时取消订阅</li>
                            <li><i class="fas fa-check"></i> 支持多设备学习</li>
                            <li><i class="fas fa-check"></i> 学习社群接入</li>
                        </ul>
                        <a href="#" class="btn btn-outline btn-block">选择此方案</a>
                    </div>
                    <div class="plan-card featured-plan">
                        <div class="plan-badge">最受欢迎</div>
                        <div class="plan-header">
                            <h3>年度会员</h3>
                            <div class="plan-price">
                                <span class="currency">¥</span>
                                <span class="amount">799</span>
                                <span class="period">/年</span>
                            </div>
                            <div class="plan-save">节省 33%</div>
                        </div>
                        <ul class="plan-features">
                            <li><i class="fas fa-check"></i> 无限访问所有课程</li>
                            <li><i class="fas fa-check"></i> 包含所有新课程</li>
                            <li><i class="fas fa-check"></i> 支持多设备学习</li>
                            <li><i class="fas fa-check"></i> 专属VIP社群</li>
                            <li><i class="fas fa-check"></i> 每月直播问答</li>
                        </ul>
                        <a href="#" class="btn btn-primary btn-block">选择此方案</a>
                    </div>
                </div>
                <div class="subscription-guarantee">
                    <i class="fas fa-shield-alt"></i>
                    <p>7天无理由退款保证，零风险尝试</p>
                </div>
            </div>
        </section>

        <!-- CTA Section -->
        <section class="cta-section">
            <div class="cta-content">
                <h2>准备好提升你的AI技能了吗？</h2>
                <p>加入超过5000名学员的行列，使用AI工具提升工作效率、创造力和职业竞争力</p>
                <a href="#" class="btn btn-primary btn-large">立即开始</a>
            </div>
        </section>
    </main>

    <footer class="main-footer">
        <div class="footer-content">
            <div class="footer-logo">
                <div class="logo">AI<span>学堂</span></div>
                <p>让AI技能学习变得简单易行</p>
            </div>
            <div class="footer-links">
                <div class="footer-column">
                    <h3>课程</h3>
                    <ul>
                        <li><a href="#">全部课程</a></li>
                        <li><a href="#">基础入门</a></li>
                        <li><a href="#">进阶提升</a></li>
                        <li><a href="#">学习路径</a></li>
                    </ul>
                </div>
                <div class="footer-column">
                    <h3>关于我们</h3>
                    <ul>
                        <li><a href="#">团队介绍</a></li>
                        <li><a href="#">讲师招募</a></li>
                        <li><a href="#">合作咨询</a></li>
                        <li><a href="#">常见问题</a></li>
                    </ul>
                </div>
                <div class="footer-column">
                    <h3>资源</h3>
                    <ul>
                        <li><a href="#">学习社区</a></li>
                        <li><a href="#">AI工具库</a></li>
                        <li><a href="#">免费教程</a></li>
                        <li><a href="#">学员作品</a></li>
                    </ul>
                </div>
            </div>
            <div class="footer-newsletter">
                <h3>订阅更新</h3>
                <p>获取最新课程、活动与AI技术动态</p>
                <form class="newsletter-form">
                    <input type="email" placeholder="你的邮箱地址" required>
                    <button type="submit" class="btn btn-primary">订阅</button>
                </form>
                <div class="social-links">
                    <a href="#"><i class="fab fa-weixin"></i></a>
                    <a href="#"><i class="fab fa-weibo"></i></a>
                    <a href="#"><i class="fab fa-zhihu"></i></a>
                    <a href="#"><i class="fab fa-bilibili"></i></a>
                </div>
            </div>
        </div>
        <div class="footer-bottom">
            <div class="copyright">© 2023 AI学堂. 保留所有权利</div>
            <div class="footer-legal">
                <a href="#">隐私政策</a>
                <a href="#">服务条款</a>
                <a href="#">退款政策</a>
            </div>
        </div>
    </footer>

    <script src="js/script.js"></script>
</body>
</html>
"""

# Create CSS file
css_content = """
/* Variables */
:root {
    /* Colors */
    --color-black: #0a0a0a;
    --color-dark: #121212;
    --color-gray-900: #1a1a1a;
    --color-gray-800: #2a2a2a;
    --color-gray-700: #333333;
    --color-gray-600: #666666;
    --color-gray-500: #888888;
    --color-gray-400: #999999;
    --color-gray-300: #cccccc;
    --color-gray-200: #dddddd;
    --color-gray-100: #eeeeee;
    --color-white: #ffffff;
    
    --color-primary: #ff4d4d;
    --color-primary-dark: #cc3333;
    --color-primary-light: #ff6666;
    
    --color-secondary: #333333;
    --color-accent: #8a2be2;
    
    /* Typography */
    --font-primary: 'Noto Sans SC', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
    
    /* Spacing */
    --space-xs: 0.25rem;
    --space-s: 0.5rem;
    --space-m: 1rem;
    --space-l: 1.5rem;
    --space-xl: 2rem;
    --space-xxl: 3rem;
    
    /* Border Radius */
    --radius-s: 4px;
    --radius-m: 8px;
    --radius-l: 12px;
    --radius-xl: 20px;
    
    /* Shadows */
    --shadow-sm: 0 2px 8px rgba(0, 0, 0, 0.15);
    --shadow-md: 0 4px 16px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 8px 30px rgba(0, 0, 0, 0.15);
    --shadow-inner: inset 0 2px 6px rgba(0, 0, 0, 0.15);
    
    /* Transitions */
    --transition-fast: 0.2s ease;
    --transition-medium: 0.3s ease;
    --transition-slow: 0.5s ease;
    
    /* Z-index */
    --z-index-dropdown: 100;
    --z-index-sticky: 200;
    --z-index-fixed: 300;
    --z-index-modal-backdrop: 400;
    --z-index-modal: 500;
    --z-index-popover: 600;
    --z-index-tooltip: 700;
    --z-index-preloader: 9999;
}

/* Base Styles */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

html {
    font-size: 16px;
    scroll-behavior: smooth;
}

body {
    font-family: var(--font-primary);
    background-color: var(--color-black);
    color: var(--color-white);
    line-height: 1.6;
    overflow-x: hidden;
    -webkit-font-smoothing: antialiased;
    -moz-osx-font-smoothing: grayscale;
}

h1, h2, h3, h4, h5, h6 {
    margin: 0 0 1rem;
    line-height: 1.3;
    font-weight: 700;
}

p {
    margin: 0 0 1.5rem;
}

a {
    color: var(--color-white);
    text-decoration: none;
    transition: color var(--transition-fast);
}

a:hover {
    color: var(--color-primary);
}

ul, ol {
    list-style: none;
}

img {
    max-width: 100%;
    height: auto;
}

button {
    cursor: pointer;
    font-family: var(--font-primary);
}

section {
    padding: 5rem 0;
}

.section {
    padding: 6rem 0;
}

.section-header {
    text-align: center;
    max-width: 800px;
    margin: 0 auto 4rem;
}

.section-title {
    font-size: 2.5rem;
    margin-bottom: 1rem;
}

.section-subtitle {
    color: var(--color-gray-400);
    font-size: 1.2rem;
}

.btn {
    display: inline-flex;
    align-items: center;
    justify-content: center;
    padding: 0.75rem 1.5rem;
    border-radius: var(--radius-m);
    font-weight: 500;
    text-align: center;
    cursor: pointer;
    transition: all var(--transition-fast);
    border: none;
}

.btn-primary {
    background: var(--color-primary);
    color: var(--color-white);
}

.btn-primary:hover {
    background: var(--color-primary-dark);
    color: var(--color-white);
    transform: translateY(-2px);
}

.btn-outline {
    background: transparent;
    color: var(--color-white);
    border: 2px solid var(--color-gray-600);
}

.btn

SyntaxError: unterminated string literal (detected at line 1) (3608273210.py, line 1)